In [46]:
import requests
import asyncio
import json
import websockets

BASE_URL = "http://localhost:8000" 

def add_user(firebase_uid: str, name: str):
    user_model = {"firebase_uid": firebase_uid, "name": name}
    response = requests.post(f"{BASE_URL}/users/", json=user_model)
    return response.status_code, response.json()

def get_users():
    response = requests.get(f"{BASE_URL}/users/")
    return response.status_code, response.json()

def get_user(id: str):
    response = requests.get(f"{BASE_URL}/users/{id}")
    return response.status_code, response.json()

def get_user_name(name: str):
    response = requests.get(f"{BASE_URL}/users/name/{name}")
    return response.status_code, response.json()

def update_user(id: str, name: str):
    response = requests.put(f"{BASE_URL}/users/{id}", json={"name": name})
    return response.status_code, response.json()

def delete_user(id: str):
    response = requests.delete(f"{BASE_URL}/users/{id}")
    return response.status_code

def get_games():
    response = requests.get(f"{BASE_URL}/games/")
    return response.status_code, response.json()

def create_game(name: str, game_type: str):
    response = requests.post(f"{BASE_URL}/games/", json={"name": name, "type": game_type})
    return response.status_code, response.json()

def add_user_to_game(game_id: str, user_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/add_user/{user_id}")
    return response.status_code, response.json()

def remove_user_from_game(game_id: str, user_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/remove_user/{user_id}")
    return response.status_code, response.json()

def start_game(game_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/start")
    return response.status_code

def delete_game(game_id: str):
    response = requests.delete(f"{BASE_URL}/games/{game_id}")
    return response.status_code

async def listen(game_id):
    uri = f"ws://localhost:8000/game/ws/{game_id}"  # Replace with your server URL
    async with websockets.connect(uri) as websocket:
        print("Connected to WebSocket")
        while True:
            message = await websocket.recv()
            data = json.loads(message)
            print(data)

def get_active_games():
    response = requests.get(f"{BASE_URL}/games/active")
    return response.status_code, response.json()

def get_active_game_debug(game_id: str):
    response = requests.get(f"{BASE_URL}/games/active/{game_id}/debug")
    return response.status_code, response.json()

def play_turn(game_id: str, player_id: str, turn_type: int, sender_id: str, receiver_id: str, cards: list[str]):
    suit_dict = {"J":0, "C":1, "D":2, "H":3, "S":4}
    rank_dict = {"0":0, "A":1, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "T":10, "J":11, "Q":12, "K":13}
    turn = {"player":player_id, "transactions":[{"sender":sender_id, "receiver":receiver_id, "card":{"rank":rank_dict[card[0]], "suit":suit_dict[card[1]]}, "success":True} for card in cards], "type":turn_type}
    response = requests.patch(f"{BASE_URL}/games/{game_id}/play", json=turn)
    return response.status_code

In [47]:
# Create vietcong game
_, game_dict = create_game("vietcongtest", "vietcong")
print(game_dict["id"])

683b6afde1cdd0e56b524ce0


In [48]:
ids = []
for user in ["vietcongp1", "vietcongp2", "vietcongp3", "vietcongp4"]:
    status, resp = add_user(user, user)
    if 'id' in resp:
        ids.append(resp['id'])
    else:
        # Fallback: try to fetch the existing user's info
        status, user_obj = get_user_name(user)
        ids.append(user_obj['id'])
print(ids)


['683a992ed4b1bfdc5b2faa4c', '683a992ed4b1bfdc5b2faa4d', '683a992ed4b1bfdc5b2faa4e', '683a992ed4b1bfdc5b2faa4f']


In [49]:
for id in ids:
    add_user_to_game(game_dict["id"], id)

In [50]:
start_game(game_dict["id"])
print(game_dict["id"])

683b6afde1cdd0e56b524ce0


In [56]:
print(get_active_game_debug(game_dict["id"]))

(200, ['683a992ed4b1bfdc5b2faa4c: 5H', '683a992ed4b1bfdc5b2faa4c: 8D', '683a992ed4b1bfdc5b2faa4c: 4H', '683a992ed4b1bfdc5b2faa4c: TH', '683a992ed4b1bfdc5b2faa4c: 8C', '683a992ed4b1bfdc5b2faa4c: 9C', '683a992ed4b1bfdc5b2faa4c: JD', '683a992ed4b1bfdc5b2faa4c: 6S', '683a992ed4b1bfdc5b2faa4c: 7S', '683a992ed4b1bfdc5b2faa4c: 6D', '683a992ed4b1bfdc5b2faa4c: 2D', '683a992ed4b1bfdc5b2faa4c: 2C', '683a992ed4b1bfdc5b2faa4c: AC', '683a992ed4b1bfdc5b2faa4d: TS', '683a992ed4b1bfdc5b2faa4d: 7C', '683a992ed4b1bfdc5b2faa4d: 2H', '683a992ed4b1bfdc5b2faa4d: QD', '683a992ed4b1bfdc5b2faa4d: 4D', '683a992ed4b1bfdc5b2faa4d: 8H', '683a992ed4b1bfdc5b2faa4d: JS', '683a992ed4b1bfdc5b2faa4d: AD', '683a992ed4b1bfdc5b2faa4d: KC', '683a992ed4b1bfdc5b2faa4d: 9H', '683a992ed4b1bfdc5b2faa4d: TD', '683a992ed4b1bfdc5b2faa4d: 4C', '683a992ed4b1bfdc5b2faa4d: 2S', '683a992ed4b1bfdc5b2faa4e: JH', '683a992ed4b1bfdc5b2faa4e: 3D', '683a992ed4b1bfdc5b2faa4e: 5C', '683a992ed4b1bfdc5b2faa4e: QC', '683a992ed4b1bfdc5b2faa4e: QS', '

In [57]:
print(play_turn(game_dict["id"], ids[3], 0, ids[3], "pile", ["3S"]))

400
